In [1]:
import jitr
from numba import njit
import numpy as np

In [2]:
@njit
def V(x, a, b):
    return a + b * x**2


@njit
def Vnl_sym(x, y, a, b):
    return a * x * y + b * (x - y) ** 2 / (y**2 + x**2)


@njit
def Vnl_asym(x, y, a, b):
    return a * x * y + b * x**2 / (y**2 + 2)

In [3]:
a = np.pi
solver = jitr.quadrature.Kernel(30, basis="Legendre")

In [4]:
analytic = 37.2894619874794
quadrature = solver.integrate_local(V, a, (2, 3))
print(f"quadrature = {quadrature:.6f}, analanalytic = {analytic:.6f}")
np.testing.assert_almost_equal(quadrature, analytic)

quadrature = 37.289462, analanalytic = 37.289462


In [5]:
analytic = 18.298993686404124
quadrature = solver.double_integrate_nonlocal(
    Vnl_asym, a, is_symmetric=False, args=(1.0 / 16, 2)
)
print(f"quadrature = {quadrature:.6f}, analanalytic = {analytic:.6f}")
np.testing.assert_almost_equal(quadrature, analytic)

quadrature = 18.298994, analanalytic = 18.298994


In [6]:
analytic = 48.58040025635857
quadrature = solver.double_integrate_nonlocal(
    Vnl_sym, 2 * np.pi, is_symmetric=False, args=(1.0 / 16, 2)
)
print(f"quadrature = {quadrature:.6f}, analanalytic = {analytic:.6f}")
np.testing.assert_almost_equal(quadrature, analytic, decimal=3)

quadrature = 48.580168, analanalytic = 48.580400


In [7]:
quadrature = solver.double_integrate_nonlocal(
    Vnl_sym, 2 * np.pi, is_symmetric=True, args=(1.0 / 16, 2)
)
print(f"quadrature = {quadrature:.6f}, analanalytic = {analytic:.6f}")
np.testing.assert_almost_equal(quadrature, analytic, decimal=3)

quadrature = 48.580168, analanalytic = 48.580400


In [8]:
lower = solver.weight_matrix[solver.lower_mask] * Vnl_sym(
    solver.Xn[solver.lower_mask] * 2 * np.pi,
    solver.Xm[solver.lower_mask] * 2 * np.pi,
    1 / 16,
    2,
)
upper = solver.weight_matrix[solver.upper_mask] * Vnl_sym(
    solver.Xn[solver.upper_mask] * 2 * np.pi,
    solver.Xm[solver.upper_mask] * 2 * np.pi,
    1 / 16,
    2,
)
full = solver.weight_matrix * Vnl_sym(
    solver.Xn * 2 * np.pi, solver.Xm * 2 * np.pi, 1 / 16, 2
)
full_sum = np.sum(full) * 4 * np.pi**2
np.testing.assert_allclose(
    np.diag(solver.weight_matrix) - solver.quadrature.weights**2, 0, atol=1e-7
)
x = np.zeros((30, 30))
x[solver.lower_mask] = lower
x[solver.upper_mask] = upper
np.testing.assert_allclose(x - full, 0.0, atol=1e-7)
np.testing.assert_allclose(np.sum(full), np.sum(lower) + np.sum(upper))